d ##### Usecase:Get sales data by departments
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for departments per day
###### Problem statement-2:Get sales data for departments per month
###### Problem statement-3:Get sales data for departments per year
###### Problem statement-4:Get monthly sales data from TOP5 revenue generatin departments
###### Problem statement-5:Get third and final quarters of 2013 sales data from TOP5 revenue generating products for each month with rankings
###### Problem statement-6:Get sales data for Men's product
###### Problem statement-7:Get monthly sales data from kids and women categories for first two quarters of 2014 
###### Problem statement-8::Get monthly sales data of TOP10 revenue generating Nike and adidas products with rankings
###### Problem statement-9:Get quarterly sales data of golf products for 2013 final quarter
###### Problem statement-10:Get sales data of TOP3 Apparel products for each year

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count,substring
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[1]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database retail_db")
hiveContext.sql("use retail_db")

In [4]:
hiveContext.sql("create table orders(order_id int,order_date string,order_customer_id int, order_status string) row format delimited fields terminated by ','")
hiveContext.sql("create table order_items(order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float) \ 
                   row format delimited fields terminated by ','")
hiveContext.sql("create table customers(customer_id int, customer_fname string, customer_lname string, customer_email string,customer_password string,customer_street string,customer_city string, \ 
                 customer_state string, customer_zipcode string) row format delimited fields terminated by ';'")
hiveContext.sql("create table products(product_id int, product_category_id int, product_name string, product_desc string, product_price float, product_image string) \ 
                                                           row format delimited fields terminated by ';'")
hiveContext.sql("create table departments(department_id int, department_name string) row format delimited fields terminated by ','")
hiveContext.sql("create table categories(category_id int, category_department_id  int, category_name string) row format delimited fields terminated by ','")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Orders.txt' into table orders")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Order_Items.txt' into table order_items")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Customers.txt' into table customers")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Products.txt' into table products")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Departments.txt' into table departments")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Categories.txt' into table categories")                

In [6]:

hiveContext.sql("show tables").show()

+---------+-----------+-----------+
 database| tableName|isTemporary|
+---------+-----------+-----------+
retail_db| categories| false|
retail_db| customers| false|
retail_db|departments| false|
retail_db|order_items| false|
retail_db| orders| false|
retail_db| products| false|
+---------+-----------+-----------+

In [7]:
hiveContext.sql("select * from orders").show(5,False)

+--------+-----------------------+-----------------+---------------+
order_id|order_date |order_customer_id|order_status |
+--------+-----------------------+-----------------+---------------+
1 |2013-07-25 00:00:00.000|11599 |CLOSED |
2 |2013-07-25 00:00:00.000|256 |PENDING_PAYMENT|
3 |2013-07-25 00:00:00.000|12111 |COMPLETE |
4 |2013-07-25 00:00:00.000|8827 |CLOSED |
5 |2013-07-25 00:00:00.000|11318 |COMPLETE |
+--------+-----------------------+-----------------+---------------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from orders").printSchema()

root
-- order_id: string (nullable = true)
-- order_date: string (nullable = true)
-- order_customer_id: string (nullable = true)
-- order_status: string (nullable = true)

In [9]:
hiveContext.sql("select * from order_items").show(5,False)


+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [10]:
hiveContext.sql("select * from order_items").printSchema()

root
-- order_item_id: string (nullable = true)
-- order_item_order_id: string (nullable = true)
-- order_item_product_id: string (nullable = true)
-- order_item_quantity: string (nullable = true)
-- order_item_subtotal: string (nullable = true)
-- order_item_product_price: string (nullable = true)

In [11]:
hiveContext.sql("select * from customers").show(5,False)

+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street |customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
1 |Richard |Hernandez |XXXXXXXXX |XXXXXXXXX |6303 Heather Plaza |Brownsville |TX |78521 |
2 |Mary |Barrett |XXXXXXXXX |XXXXXXXXX |9526 Noble Embers Ridge|Littleton |CO |80126 |
3 |Ann |Smith |XXXXXXXXX |XXXXXXXXX |3422 Blue Pioneer Bend |Caguas |PR |00725 |
4 |Mary |Jones |XXXXXXXXX |XXXXXXXXX |8324 Little Common |San Marcos |CA |92069 |
5 |Robert |Hudson |XXXXXXXXX |XXXXXXXXX |10 Crystal River Mall |Caguas |PR |00725 |
+-----------+--------------+--------------+--------------+-----------------+-----------------------+-------------+--------------+----------------+
only showing top 5 rows

In [12]:
hiveContext.sql("select * from customers").printSchema()

root
-- customer_id: string (nullable = true)
-- customer_fname: string (nullable = true)
-- customer_lname: string (nullable = true)
-- customer_email: string (nullable = true)
-- customer_password: string (nullable = true)
-- customer_street: string (nullable = true)
-- customer_city: string (nullable = true)
-- customer_state: string (nullable = true)
-- customer_zipcode: string (nullable = true)

In [13]:
hiveContext.sql("select * from products").show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
product_id|product_category_id| product_name|product_description|product_price| product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
 1| 2|Quest Q64 10 FT. ...| null| 59.98|http://images.acm...|
 2| 2|Under Armour Men'...| null| 129.99|http://images.acm...|
 3| 2|Under Armour Men'...| null| 89.99|http://images.acm...|
 4| 2|Under Armour Men'...| null| 89.99|http://images.acm...|
 5| 2|Riddell Youth Rev...| null| 199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 rows

In [14]:
hiveContext.sql("select * from products").printSchema()

root
-- product_id: string (nullable = true)
-- product_category_id: string (nullable = true)
-- product_name: string (nullable = true)
-- product_description: string (nullable = true)
-- product_price: string (nullable = true)
-- product_image: string (nullable = true)

In [15]:
hiveContext.sql("select * from categories").show(5)

+-----------+----------------------+-------------------+
category_id|category_department_id| category_name|
+-----------+----------------------+-------------------+
 1| 2| Football|
 2| 2| Soccer|
 3| 2|Baseball & Softball|
 4| 2| Basketball|
 5| 2| Lacrosse|
+-----------+----------------------+-------------------+
only showing top 5 rows

In [16]:
hiveContext.sql("select * from categories").printSchema()

root
-- category_id: string (nullable = true)
-- category_department_id: string (nullable = true)
-- category_name: string (nullable = true)

In [17]:
hiveContext.sql("select * from departments").show(5)

+-------------+---------------+
department_id|department_name|
+-------------+---------------+
 2| Fitness|
 3| Footwear|
 4| Apparel|
 5| Golf|
 6| Outdoors|
+-------------+---------------+
only showing top 5 rows

In [18]:
hiveContext.sql("select * from departments").printSchema()

root
-- department_id: string (nullable = true)
-- department_name: string (nullable = true)

###### Problem statement-1:Get sales data for departments per day

In [20]:
departmentsSalesDataPerDay = hiveContext.sql('''select substring(otl.order_date,0,10) as orderdate,dept.department_name,
                                   round(sum(oitl.order_item_subtotal),2) as deptrevenue_perday,  
                                   count(oitl.order_item_subtotal) as NumberOforders_perday, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   group by orderdate,dept.department_name 
                                   order by orderdate,dept.department_name''')
departmentsSalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/productsSalesDataPerDay", compression="none", header ='true')
departmentsSalesDataPerDay.show(20,False)

+----------+---------------+------------------+---------------------+---------------+
orderdate |department_name|deptrevenue_perday|NumberOforders_perday|avgsales_perday|
+----------+---------------+------------------+---------------------+---------------+
2013-07-25|Apparel |13148.31 |86 |152.89 |
2013-07-25|Fan Shop |34392.89 |136 |252.89 |
2013-07-25|Fitness |564.91 |4 |141.23 |
2013-07-25|Footwear |9355.06 |29 |322.59 |
2013-07-25|Golf |8299.24 |61 |136.05 |
2013-07-25|Outdoors |2393.42 |23 |104.06 |
2013-07-26|Apparel |31525.69 |197 |160.03 |
2013-07-26|Fan Shop |67227.82 |258 |260.57 |
2013-07-26|Fitness |671.91 |11 |61.08 |
2013-07-26|Footwear |15947.3 |67 |238.02 |
2013-07-26|Golf |17213.49 |124 |138.82 |
2013-07-26|Outdoors |3933.96 |37 |106.32 |
2013-07-27|Apparel |17617.74 |118 |149.3 |
2013-07-27|Fan Shop |56663.48 |218 |259.92 |
2013-07-27|Fitness |347.94 |4 |86.99 |
2013-07-27|Footwear |11276.91 |40 |281.92 |
2013-07-27|Golf |12353.94 |96 |128.69 |
2013-07-27|Outdoors |2814.33 |27 |104.23 |
2013-07-28|Apparel |24536.75 |150 |163.58 |
2013-07-28|Fan Shop |41975.36 |158 |265.67 |
+----------+---------------+------------------+---------------------+---------------+
only showing top 20 rows

###### Problem statement-2: Get sales data for departments per month

In [22]:
departmentsSalesDataPerMonth = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,dept.department_name,
                                   round(sum(oitl.order_item_subtotal),2) as deptrevenue_permonth,  
                                   count(oitl.order_item_subtotal) as NumberOforders_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   group by ordermonth,dept.department_name 
                                   order by ordermonth,dept.department_name''')
departmentsSalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerMonth", compression="none", header ='true')
departmentsSalesDataPerMonth.show(20,False)

+----------+---------------+--------------------+-----------------------+-----------------+
ordermonth|department_name|deptrevenue_permonth|NumberOforders_permonth|avgsales_permonth|
+----------+---------------+--------------------+-----------------------+-----------------+
2013-07 |Apparel |163108.27 |1028 |158.67 |
2013-07 |Fan Shop |389462.07 |1491 |261.21 |
2013-07 |Fitness |5622.15 |50 |112.44 |
2013-07 |Footwear |84624.92 |321 |263.63 |
2013-07 |Golf |101087.26 |723 |139.82 |
2013-07 |Outdoors |20877.52 |208 |100.37 |
2013-08 |Apparel |598981.21 |3809 |157.25 |
2013-08 |Fan Shop |1410128.64 |5508 |256.01 |
2013-08 |Fitness |23719.61 |173 |137.11 |
2013-08 |Footwear |321443.79 |1190 |270.12 |
2013-08 |Golf |391535.73 |2800 |139.83 |
2013-08 |Outdoors |82849.72 |788 |105.14 |
2013-09 |Apparel |615739.55 |3941 |156.24 |
2013-09 |Fan Shop |1452927.68 |5638 |257.7 |
2013-09 |Fitness |24377.45 |187 |130.36 |
2013-09 |Footwear |369264.45 |1331 |277.43 |
2013-09 |Golf |388137.44 |2813 |137.98 |
2013-09 |Outdoors |84080.7 |784 |107.25 |
2013-10 |Apparel |562546.1 |3616 |155.57 |
2013-10 |Fan Shop |1297740.34 |5027 |258.15 |
+----------+---------------+--------------------+-----------------------+-----------------+
only showing top 20 rows

###### Problem statement-3: Get sales data  for departments per year

In [24]:
departmentsSalesDataPerYear = hiveContext.sql('''select substring(otl.order_date,0,4) as orderyear,dept.department_name,
                                   round(sum(oitl.order_item_subtotal),2) as deptrevenue_peryear,  
                                   count(oitl.order_item_subtotal) as NumberOforders_peryear, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_peryear 
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   group by orderyear,dept.department_name 
                                   order by orderyear,dept.department_name''')
departmentsSalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerYear", compression="none", header ='true')
departmentsSalesDataPerYear.show(20,False)

+---------+---------------+-------------------+----------------------+----------------+
orderyear|department_name|deptrevenue_peryear|NumberOforders_peryear|avgsales_peryear|
+---------+---------------+-------------------+----------------------+----------------+
2013 |Apparel |3228557.53 |20665 |156.23 |
2013 |Fan Shop |7615003.64 |29503 |258.11 |
2013 |Fitness |123587.89 |951 |129.96 |
2013 |Footwear |1789823.74 |6564 |272.67 |
2013 |Golf |2058230.89 |14846 |138.64 |
2013 |Outdoors |438985.38 |4104 |106.97 |
2014 |Apparel |4095142.67 |26132 |156.71 |
2014 |Fan Shop |9492762.24 |36829 |257.75 |
2014 |Fitness |156456.25 |1171 |133.61 |
2014 |Footwear |2216675.03 |7961 |278.44 |
2014 |Golf |2550797.33 |18374 |138.83 |
2014 |Outdoors |556597.34 |5098 |109.18 |
+---------+---------------+-------------------+----------------------+----------------+

###### Problem statement-4:Get sales data from TOP5 reveue generating departments

In [26]:
departmentsSalesDatafortop5dept = hiveContext.sql('''select dept.department_name,
                                   round(sum(oitl.order_item_subtotal),2) as deptrevenue,sum(oitl.order_item_quantity) order_quantity
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   group by dept.department_name
                                   order by deptrevenue desc''')
departmentsSalesDatafortop5dept.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDatafortop5dept", compression="none", header ='true')
departmentsSalesDatafortop5dept.show(20,False)

+---------------+-------------+--------------+
department_name|deptrevenue |order_quantity|
+---------------+-------------+--------------+
Fan Shop |1.710776588E7|105636.0 |
Apparel |7323700.2 |95980.0 |
Golf |4609028.22 |99297.0 |
Footwear |4006498.77 |43400.0 |
Outdoors |995582.72 |25575.0 |
Fitness |280044.14 |5870.0 |
+---------------+-------------+--------------+

###### Problem statement-5:Get third and final quarters of 2013 sales data from TOP5 revenue generating products for each month with rankings

In [28]:
departmentSalesDatafortop5perquart = hiveContext.sql('''select * from (select substring(otl.order_date,0,7) as ordermonth,dept.department_name,
                                   round(sum(oitl.order_item_subtotal),2) as deptrevenue,
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl,categories catg, departments dept where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id and catg.category_id = pdtl.product_category_id 
                                   and dept.department_id =  catg.category_department_id and CAST(otl.order_date AS DATE)>='2013-07-01' 
                                   and CAST(otl.order_date AS DATE)<='2013-12-31'
                                   group by substring(otl.order_date,0,7),dept.department_name) t
                                    ORDER BY 1,3 desc''')
departmentSalesDatafortop5perquart.coalesce(1).write.csv("/FileStore/tables/departmentSalesDatafortop5perquart", compression="none", header ='true')
departmentSalesDatafortop5perquart.show(36,False)

+----------+---------------+-----------+-------------+
ordermonth|department_name|deptrevenue|ranking_order|
+----------+---------------+-----------+-------------+
2013-07 |Fan Shop |389462.07 |1 |
2013-07 |Apparel |163108.27 |2 |
2013-07 |Golf |101087.26 |3 |
2013-07 |Footwear |84624.92 |4 |
2013-07 |Outdoors |20877.52 |5 |
2013-07 |Fitness |5622.15 |6 |
2013-08 |Fan Shop |1410128.64 |1 |
2013-08 |Apparel |598981.21 |2 |
2013-08 |Golf |391535.73 |3 |
2013-08 |Footwear |321443.79 |4 |
2013-08 |Outdoors |82849.72 |5 |
2013-08 |Fitness |23719.61 |6 |
2013-09 |Fan Shop |1452927.68 |1 |
2013-09 |Apparel |615739.55 |2 |
2013-09 |Golf |388137.44 |3 |
2013-09 |Footwear |369264.45 |4 |
2013-09 |Outdoors |84080.7 |5 |
2013-09 |Fitness |24377.45 |6 |
2013-10 |Fan Shop |1297740.34 |1 |
2013-10 |Apparel |562546.1 |2 |
2013-10 |Golf |362129.11 |3 |
2013-10 |Footwear |300716.9 |4 |
2013-10 |Outdoors |77542.62 |5 |
2013-10 |Fitness |23925.54 |6 |
2013-11 |Fan Shop |1576613.49 |1 |
2013-11 |Apparel |679301.42 |2 |
2013-11 |Golf |427289.86 |3 |
2013-11 |Footwear |371817.8 |4 |
2013-11 |Outdoors |90549.99 |5 |
2013-11 |Fitness |23083.47 |6 |
2013-12 |Fan Shop |1488131.42 |1 |
2013-12 |Apparel |608880.98 |2 |
2013-12 |Golf |388051.49 |3 |
2013-12 |Footwear |341955.88 |4 |
2013-12 |Outdoors |83084.83 |5 |
2013-12 |Fitness |22859.67 |6 |
+----------+---------------+-----------+-------------+

###### Problem statement-6:Get sales data for Men's product

In [30]:
departmentsSalesDataforMensprod = hiveContext.sql('''select dept.department_name,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as productrevenue,sum(oitl.order_item_quantity) order_quantity
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   WHERE pdtl.product_name LIKE '%Men%'
                                   group by dept.department_name,pdtl.product_name
                                   order by productrevenue desc''')
departmentsSalesDataforMensprod.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataforMensprod", compression="none", header ='true')
departmentsSalesDataforMensprod .show(20,False)

+---------------+---------------------------------------------+--------------+--------------+
department_name|product_name |productrevenue|order_quantity|
+---------------+---------------------------------------------+--------------+--------------+
Footwear |Nike Men's Free 5.0+ Running Shoe |3667633.2 |36680.0 |
Golf |Nike Men's Dri-FIT Victory Golf Polo |3147800.0 |62956.0 |
Apparel |Nike Men's CJ Elite 2 TD Football Cleat |2891757.54 |22246.0 |
Fan Shop |O'Brien Men's Neoprene Life Vest |2888993.94 |57803.0 |
Fitness |adidas Men's F10 Messi TRX FG Soccer Cleat |56330.61 |939.0 |
Fitness |Nike Men's Comfort 2 Slide |44585.09 |991.0 |
Footwear |Under Armour Men's Compression EV SL Slide |36576.87 |813.0 |
Golf |Columbia Men's PFG Anchor Tough T-Shirt |27840.0 |928.0 |
Golf |Nike Men's Deutschland Weltmeister Winners Bl|27210.0 |907.0 |
Golf |adidas Men's Germany Black Crest Away Tee |21475.0 |859.0 |
Fitness |Under Armour Men's Tech II T-Shirt |20866.65 |835.0 |
Footwear |Nike Men's Free TR 5.0 TB Training Shoe |20597.94 |206.0 |
Outdoors |Hirzl Men's Hybrid Golf Glove |12951.36 |864.0 |
Golf |Nike Men's Kobe IX Elite Low Basketball Shoe |12599.37 |63.0 |
Outdoors |Merrell Men's All Out Flash Trail Running Sho|9349.15 |85.0 |
Fitness |Nike Men's Fingertrap Max Training Shoe |7999.36 |64.0 |
+---------------+---------------------------------------------+--------------+--------------+

###### Problem statement-7:Get monthly sales data from kids and women categories for first two quarters of 2014

In [32]:
departmentsSalesDataforkidwomen = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,dept.department_name,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as productrevenue,sum(oitl.order_item_quantity) order_quantity
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   WHERE(pdtl.product_name LIKE '%Kid%' or pdtl.product_name LIKE '%Women%')and
                                   (CAST(otl.order_date AS DATE)>="2014-01-01" AND CAST(otl.order_date AS DATE)<='2014-06-30')
                                   group by ordermonth,dept.department_name,pdtl.product_name
                                   order by ordermonth,productrevenue desc''')
departmentsSalesDataforkidwomen.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataforkidwomen", compression="none", header ='true')
departmentsSalesDataforkidwomen.show(30,False)

+----------+---------------+---------------------------------------------+--------------+--------------+
ordermonth|department_name|product_name |productrevenue|order_quantity|
+----------+---------------+---------------------------------------------+--------------+--------------+
2014-01 |Fan Shop |Diamondback Women's Serene Classic Comfort Bi|341677.22 |1139.0 |
2014-01 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |4160.0 |64.0 |
2014-01 |Footwear |Under Armour Women's Micro G Skulpt Running S|3957.84 |72.0 |
2014-01 |Outdoors |LIJA Women's Mid-Length Panel Golf Shorts |3000.0 |30.0 |
2014-01 |Footwear |Under Armour Women's Ignite Slide |2655.17 |83.0 |
2014-01 |Footwear |Nike Women's Tempo Shorts |2280.0 |76.0 |
2014-01 |Footwear |Under Armour Kids' Mercenary Slide |1987.29 |71.0 |
2014-01 |Outdoors |Glove It Women's Imperial Golf Glove |1919.04 |96.0 |
2014-01 |Fitness |Nike Kids' Grade School KD VI Basketball Shoe|1899.81 |19.0 |
2014-01 |Outdoors |Merrell Women's Grassbow Sport Hiking Shoe |1899.81 |19.0 |
2014-01 |Outdoors |Glove It Women's Mod Oval 3-Zip Carry All Gol|1869.15 |85.0 |
2014-01 |Footwear |Under Armour Women's Ignite PIP VI Slide |1855.42 |58.0 |
2014-01 |Footwear |Brooks Women's Ghost 6 Running Shoe |1799.8 |20.0 |
2014-01 |Fitness |adidas Kids' F5 Messi FG Soccer Cleat |1714.51 |49.0 |
2014-01 |Fan Shop |The North Face Women's Recon Backpack |1584.0 |16.0 |
2014-01 |Footwear |Nike Women's Free 5.0 TR FIT PRT 4 Training S|1329.86 |14.0 |
2014-01 |Outdoors |Top Flite Women's 2014 XL Hybrid |1319.34 |66.0 |
2014-01 |Fitness |Nike Women's Legend V-Neck T-Shirt |1250.0 |50.0 |
2014-01 |Outdoors |Glove It Women's Mod Oval Golf Glove |1139.43 |57.0 |
2014-01 |Outdoors |Hirzl Women's Soffft Flex Golf Glove |1079.4 |60.0 |
2014-01 |Outdoors |Hirzl Women's Hybrid Golf Glove |974.35 |65.0 |
2014-01 |Outdoors |TaylorMade Women's RBZ SL Rescue |909.93 |7.0 |
2014-01 |Outdoors |Cleveland Golf Women's 588 RTX CB Satin Chrom|839.93 |7.0 |
2014-01 |Outdoors |LIJA Women's Argyle Golf Polo |800.0 |10.0 |
2014-01 |Outdoors |LIJA Women's Button Golf Dress |756.0 |7.0 |
2014-01 |Outdoors |Merrell Women's Siren Mid Waterproof Hiking B|674.95 |5.0 |
2014-01 |Outdoors |Merrell Women's Grassbow Sport Waterproof Hik|649.95 |5.0 |
2014-02 |Fan Shop |Diamondback Women's Serene Classic Comfort Bi|332977.8 |1110.0 |
2014-02 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |6630.0 |102.0 |
2014-02 |Footwear |Under Armour Women's Micro G Skulpt Running S|3573.05 |65.0 |
+----------+---------------+---------------------------------------------+--------------+--------------+
only showing top 30 rows

###### Problem statement-8:Get monthly sales data of TOP10 revenue generating Nike and adidas products with rankings

In [34]:
departSalesDatafortop10 = hiveContext.sql('''select * from (select substring(otl.order_date,0,7) as ordermonth,pdtl.product_name,
                                           round(sum(oitl.order_item_subtotal),2) as deptrevenue,
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl,categories catg, departments dept where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id and catg.category_id = pdtl.product_category_id
                                   and dept.department_id =  catg.category_department_id and (pdtl.product_name LIKE '%Nike%' or pdtl.product_name LIKE '%adidas%')
                                   group by substring(otl.order_date,0,7),pdtl.product_name) t
                                   WHERE ranking_order<=10 ORDER BY 1,4''')
departSalesDatafortop10.coalesce(1).write.csv("/FileStore/tables/departSalesDatafortop10", compression="none", header ='true')
departSalesDatafortop10.show(30,False)

+----------+---------------------------------------------+-----------+-------------+
ordermonth|product_name |deptrevenue|ranking_order|
+----------+---------------------------------------------+-----------+-------------+
2013-07 |Nike Men's Free 5.0+ Running Shoe |78292.17 |1 |
2013-07 |Nike Men's Dri-FIT Victory Golf Polo |67100.0 |2 |
2013-07 |Nike Men's CJ Elite 2 TD Football Cleat |59925.39 |3 |
2013-07 |Nike Men's Comfort 2 Slide |1484.67 |4 |
2013-07 |adidas Youth Germany Black/Red Away Match Soc|1050.0 |5 |
2013-07 |Nike Women's Tempo Shorts |840.0 |6 |
2013-07 |Nike Women's Legend V-Neck T-Shirt |725.0 |7 |
2013-07 |adidas Men's F10 Messi TRX FG Soccer Cleat |659.89 |8 |
2013-07 |Nike Men's Deutschland Weltmeister Winners Bl|630.0 |9 |
2013-07 |Nike Dri-FIT Crew Sock 6 Pack |528.0 |10 |
2013-08 |Nike Men's Free 5.0+ Running Shoe |292870.71 |1 |
2013-08 |Nike Men's Dri-FIT Victory Golf Polo |261150.0 |2 |
2013-08 |Nike Men's CJ Elite 2 TD Football Cleat |231902.16 |3 |
2013-08 |adidas Youth Germany Black/Red Away Match Soc|6790.0 |4 |
2013-08 |adidas Men's F10 Messi TRX FG Soccer Cleat |6358.94 |5 |
2013-08 |Nike Men's Comfort 2 Slide |3644.19 |6 |
2013-08 |Nike Women's Tempo Shorts |2340.0 |7 |
2013-08 |Nike Men's Free TR 5.0 TB Training Shoe |2099.79 |8 |
2013-08 |adidas Men's Germany Black Crest Away Tee |2075.0 |9 |
2013-08 |Nike Dri-FIT Crew Sock 6 Pack |1958.0 |10 |
2013-09 |Nike Men's Free 5.0+ Running Shoe |340165.98 |1 |
2013-09 |Nike Men's Dri-FIT Victory Golf Polo |264900.0 |2 |
2013-09 |Nike Men's CJ Elite 2 TD Football Cleat |245421.12 |3 |
2013-09 |adidas Men's F10 Messi TRX FG Soccer Cleat |5639.06 |4 |
2013-09 |adidas Youth Germany Black/Red Away Match Soc|5390.0 |5 |
2013-09 |Nike Men's Comfort 2 Slide |2789.38 |6 |
2013-09 |Nike Women's Tempo Shorts |2760.0 |7 |
2013-09 |Nike Kids' Grade School KD VI Basketball Shoe|2599.74 |8 |
2013-09 |Nike Men's Deutschland Weltmeister Winners Bl|2370.0 |9 |
2013-09 |adidas Kids' F5 Messi FG Soccer Cleat |2204.37 |10 |
+----------+---------------------------------------------+-----------+-------------+
only showing top 30 rows

###### Problem statement-9:Get quarterly sales data of golf products for 2013 final quarter

In [36]:
golfprodSalesData2013fquart = hiveContext.sql('''select substring(otl.order_date,0,7) as ordermonth,dept.department_name,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as productrevenue,sum(oitl.order_item_quantity) order_quantity
                                   from orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id
                                   join products pdtl on oitl.order_item_product_id = pdtl.product_id
                                   join categories catg on catg.category_id = pdtl.product_category_id
                                   join departments dept on dept.department_id =  catg.category_department_id
                                   WHERE(pdtl.product_name LIKE '%Golf%')and
                                   (CAST(otl.order_date AS DATE)>="2013-10-01" AND CAST(otl.order_date AS DATE)<='2013-12-31')
                                   group by ordermonth,dept.department_name,pdtl.product_name
                                   order by ordermonth,productrevenue desc''')
golfprodSalesData2013fquart.coalesce(1).write.csv("/FileStore/tables/golfprodSalesData2013fquart", compression="none", header ='true')
golfprodSalesData2013fquart.show(30,False)

+----------+---------------+---------------------------------------------+--------------+--------------+
ordermonth|department_name|product_name |productrevenue|order_quantity|
+----------+---------------+---------------------------------------------+--------------+--------------+
2013-10 |Golf |Nike Men's Dri-FIT Victory Golf Polo |247050.0 |4941.0 |
2013-10 |Outdoors |Titleist Pro V1x Golf Balls |4319.1 |90.0 |
2013-10 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |4160.0 |64.0 |
2013-10 |Outdoors |Titleist Pro V1x High Numbers Golf Balls |2783.42 |58.0 |
2013-10 |Outdoors |Team Golf Texas Longhorns Putter Grip |1849.26 |74.0 |
2013-10 |Outdoors |Team Golf Pittsburgh Steelers Putter Grip |1799.28 |72.0 |
2013-10 |Outdoors |LIJA Women's Argyle Golf Polo |1760.0 |22.0 |
2013-10 |Outdoors |Team Golf Tennessee Volunteers Putter Grip |1749.3 |70.0 |
2013-10 |Outdoors |Team Golf St. Louis Cardinals Putter Grip |1699.32 |68.0 |
2013-10 |Outdoors |Team Golf San Francisco Giants Putter Grip |1524.39 |61.0 |
2013-10 |Outdoors |Garmin Approach S4 Golf GPS Watch |1499.95 |5.0 |
2013-10 |Outdoors |Cleveland Golf Collegiate My Custom Wedge 588|1469.93 |7.0 |
2013-10 |Outdoors |Ogio Race Golf Shoes |1359.92 |8.0 |
2013-10 |Outdoors |Glove It Women's Imperial Golf Glove |1339.33 |67.0 |
2013-10 |Outdoors |Glove It Women's Mod Oval Golf Glove |1279.36 |64.0 |
2013-10 |Outdoors |Garmin Approach S3 Golf GPS Watch |1249.95 |5.0 |
2013-10 |Outdoors |Team Golf New England Patriots Putter Grip |1224.51 |49.0 |
2013-10 |Outdoors |Glove It Urban Brick Golf Towel |1199.25 |75.0 |
2013-10 |Outdoors |Hirzl Women's Soffft Flex Golf Glove |1079.4 |60.0 |
2013-10 |Outdoors |Glove It Imperial Golf Towel |1023.36 |64.0 |
2013-10 |Outdoors |Hirzl Women's Hybrid Golf Glove |869.42 |58.0 |
2013-10 |Outdoors |MDGolf Pittsburgh Penguins Putter |799.9 |10.0 |
2013-10 |Outdoors |Hirzl Men's Hybrid Golf Glove |779.48 |52.0 |
2013-10 |Outdoors |GolfBuddy VT3 GPS Watch |599.97 |3.0 |
2013-10 |Outdoors |LIJA Women's Mid-Length Panel Golf Shorts |500.0 |5.0 |
2013-10 |Outdoors |Cleveland Golf Women's 588 RTX CB Satin Chrom|479.96 |4.0 |
2013-10 |Outdoors |LIJA Women's Button Golf Dress |108.0 |1.0 |
2013-11 |Golf |Nike Men's Dri-FIT Victory Golf Polo |291500.0 |5830.0 |
2013-11 |Outdoors |LIJA Women's Eyelet Sleeveless Golf Polo |6370.0 |98.0 |
2013-11 |Outdoors |Titleist Pro V1x High Numbers Golf Balls |3023.37 |63.0 |
+----------+---------------+---------------------------------------------+--------------+--------------+
only showing top 30 rows

###### Problem statement-10:Get sales data of TOP3 Apparel products for each year

In [38]:
apparelSalesDatatop5peryear = hiveContext.sql('''select * from (select substring(otl.order_date,0,4) as orderyear,dept.department_name,pdtl.product_name,
                                   round(sum(oitl.order_item_subtotal),2) as Totalrevenue,  
                                   count(oitl.order_item_subtotal) as NumberOforders, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales,
                                   rank() over(partition by substring(otl.order_date,0,4) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from orders otl,order_items oitl,products pdtl,categories catg, departments dept where otl.order_id=oitl.order_item_order_id
                                   and oitl.order_item_product_id = pdtl.product_id and catg.category_id = pdtl.product_category_id 
                                   and dept.department_id =  catg.category_department_id and dept.department_name = "Apparel"
                                   group by substring(otl.order_date,0,4),dept.department_name,pdtl.product_name) t 
                                   WHERE ranking_order<=5 ORDER BY 1, 7 ''')
apparelSalesDatatop5peryear.coalesce(1).write.csv("/FileStore/tables/apparelSalesDatatop5peryear", compression="none", header ='true')
apparelSalesDatatop5peryear.show(30,False)

+---------+---------------+---------------------------------------+------------+--------------+--------+-------------+
orderyear|department_name|product_name |Totalrevenue|NumberOforders|avgsales|ranking_order|
+---------+---------------+---------------------------------------+------------+--------------+--------+-------------+
2013 |Apparel |Perfect Fitness Perfect Rip Deck |1942416.21 |10797 |179.9 |1 |
2013 |Apparel |Nike Men's CJ Elite 2 TD Football Cleat|1280141.52 |9848 |129.99 |2 |
2013 |Apparel |Total Gym 1400 |5999.8 |20 |299.99 |3 |
2014 |Apparel |Perfect Fitness Perfect Rip Deck |2478726.81 |13718 |180.69 |1 |
2014 |Apparel |Nike Men's CJ Elite 2 TD Football Cleat|1611616.02 |12398 |129.99 |2 |
2014 |Apparel |Total Gym 1400 |4799.84 |16 |299.99 |3 |
+---------+---------------+---------------------------------------+------------+--------------+--------+-------------+